<a href="https://colab.research.google.com/github/jkky-98/airdeep-teamproject/blob/main/%EA%B8%B0%EC%83%81%EC%B2%AD_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install --upgrade --user nbconvert

In [1]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
import pandas as pd

#기상청_지상(종관, ASOS)시간자료_조회서비스


In [154]:
def hourWthr2csv(startDt, startHh, endDt, endHh):
    
    # Setting for URL parsing
    url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
    # pageNo = str(((numOfRow(startDt, startHh, endDt, endHh))/1000)+1)

    params = '?' + urlencode({
        quote_plus("serviceKey"): 'lq6n0tj9dXuCRO4ia6GfDNtqcolskcey90BQTo454rgEPPBLeRiDHYDgWmzHFlzYcgg6FVoDNGeynpwACMSMQw%3D%3D',      # 인증키
        quote_plus("numOfRows"): '900',          # 
        quote_plus("pageNo"): '1',              # 
        quote_plus("dataType"): "JSON",         # 
        quote_plus("dataCd"): "ASOS",           # 
        quote_plus("dateCd"): "HR",             # 
        quote_plus("startDt"): str(startDt),         # 시작 날짜
        quote_plus("startHh"): str(startHh),         # 시작 시간
        quote_plus("endDt"): str(endDt),             # 끝 날짜
        quote_plus("endHh"): str(endHh),             # 끝 시간
        quote_plus("stnIds"): "119"             # 예보지점 = 수원
        })

    # URL parsing
    req = urllib.request.Request(url + unquote(params))

    # Get Data from API
    response_body = urlopen(req).read() # get bytes data

    # Convert bytes to json
    data = json.loads(response_body)
    print(data)

    # Result
    res = pd.DataFrame(data['response']['body']['items']['item'])

    df = pd.DataFrame()
    df['tim'] = res['tm']        #시간
    df['temp'] = res['ta']        #기온
    df['humid'] = res['hm']       #습도
    df['rain'] = res['rn']        #강수량
    df['wind'] = res['wd']        #풍속
    df['sunshine'] = res['ss']    #일조:태양이 지표면을 비친 시간
    df['solar radiation'] = res['icsr'] #일사:지표면에 도달한 태양복사에너지

    df.to_csv('{}-{}_hourWthr.csv'.format(str(startDt),str(endDt)))

In [155]:
#한 달치만 가능
hourWthr2csv(startDt = '20220101', startHh = '00', endDt = '20220131', endHh = '23')

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'}, 'body': {'dataType': 'JSON', 'items': {'item': [{'tm': '2022-01-01 00:00', 'rnum': '1', 'stnId': '119', 'stnNm': '수원', 'ta': '-7.8', 'taQcflg': '', 'rn': '', 'rnQcflg': '', 'ws': '1.7', 'wsQcflg': '', 'wd': '320', 'wdQcflg': '', 'hm': '42', 'hmQcflg': '', 'pv': '1.4', 'td': '-18.5', 'pa': '1028.4', 'paQcflg': '', 'ps': '1033.7', 'psQcflg': '', 'ss': '', 'ssQcflg': '9', 'icsr': '', 'dsnw': '', 'hr3Fhsc': '', 'dc10Tca': '0', 'dc10LmcsCa': '0', 'clfmAbbrCd': '', 'lcsCh': '', 'vs': '2000', 'gndSttCd': '', 'dmstMtphNo': '', 'ts': '-6.7', 'tsQcflg': '', 'm005Te': '', 'm01Te': '', 'm02Te': '', 'm03Te': ''}, {'tm': '2022-01-01 01:00', 'rnum': '2', 'stnId': '119', 'stnNm': '수원', 'ta': '-9.0', 'taQcflg': '', 'rn': '', 'rnQcflg': '', 'ws': '0.6', 'wsQcflg': '', 'wd': '290', 'wdQcflg': '', 'hm': '48', 'hmQcflg': '', 'pv': '1.5', 'td': '-18.0', 'pa': '1028.8', 'paQcflg': '', 'ps': '1034.1', 'psQcflg': '', 'ss': '', 'ssQcfl

,tim,temp,humid,rain,wind,sunshine,solar radiation
0,2022-01-01 00:00,-7.8,42,,320,,
1,2022-01-01 01:00,-9.0,48,,290,,
2,2022-01-01 02:00,-8.8,47,,290,,
3,2022-01-01 03:00,-9.1,49,,0,,
4,2022-01-01 04:00,-10.4,59,,270,,
...,...,...,...,...,...,...,...
895,2022-02-07 07:00,-9.0,74,,0,,
896,2022-02-07 08:00,-8.4,72,,0,0.2,0.03
897,2022-02-07 09:00,-3.6,58,,0,1.0,0.53
898,2022-02-07 10:00,-2.0,52,,200,1.0,1.05


#쓰잘데기 없는거

In [123]:
# from pandas._libs.tslibs.dtypes import PeriodDtypeBase
# import datetime as dt

# def numOfRow(startDt, startHh, endDt, endHh):
#     start = str(startDt) + str(startHh)
#     end = str(endDt) + str(endHh)

#     start = dt.datetime.strptime(start, "%Y%m%d%H")
#     end = dt.datetime.strptime(end, "%Y%m%d%H")
#     # print(start, end)

#     period = int((start-end).days)
#     period *= 24

#     return period

In [130]:
# int(numOfRow(startDt = 20230101, startHh = 00, endDt = 20220228, endHh = 23)/1000)+1

7

In [7]:
# url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

# params = '?' + urlencode({
#     quote_plus("serviceKey"): 'lq6n0tj9dXuCRO4ia6GfDNtqcolskcey90BQTo454rgEPPBLeRiDHYDgWmzHFlzYcgg6FVoDNGeynpwACMSMQw%3D%3D',      # 인증키
#     quote_plus("numOfRows"): "10",          # 
#     quote_plus("pageNo"): "1",              # 
#     quote_plus("dataType"): "JSON",         # 
#     quote_plus("dataCd"): "ASOS",           # 
#     quote_plus("dateCd"): "HR",             # 
#     quote_plus("startDt"): '20230101',         # 시작 날짜
#     quote_plus("startHh"): '01',         # 시작 시간
#     quote_plus("endDt"): '20230228',             # 끝 날짜
#     quote_plus("endHh"): '01',             # 끝 시간
#     quote_plus("stnIds"): "119"             # 예보지점 = 수원
#     })

# # URL parsing
# req = urllib.request.Request(url + unquote(params))

# # Get Data from API
# response_body = urlopen(req).read() # get bytes data

# # Convert bytes to json
# data = json.loads(response_body)
# print(data)

# # Result
# res = pd.DataFrame(data['response']['body']['items']['item'])


{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'}, 'body': {'dataType': 'JSON', 'items': {'item': [{'tm': '2023-01-01 01:00', 'rnum': '1', 'stnId': '119', 'stnNm': '수원', 'ta': '-4.5', 'taQcflg': '', 'rn': '', 'rnQcflg': '', 'ws': '1.5', 'wsQcflg': '', 'wd': '140', 'wdQcflg': '', 'hm': '94', 'hmQcflg': '', 'pv': '4.1', 'td': '-5.3', 'pa': '1025.2', 'paQcflg': '', 'ps': '1030.4', 'psQcflg': '', 'ss': '', 'ssQcflg': '9', 'icsr': '', 'dsnw': '', 'hr3Fhsc': '', 'dc10Tca': '4', 'dc10LmcsCa': '0', 'clfmAbbrCd': '', 'lcsCh': '', 'vs': '600', 'gndSttCd': '', 'dmstMtphNo': '19', 'ts': '-2.8', 'tsQcflg': '', 'm005Te': '', 'm01Te': '', 'm02Te': '', 'm03Te': ''}, {'tm': '2023-01-01 02:00', 'rnum': '2', 'stnId': '119', 'stnNm': '수원', 'ta': '-4.1', 'taQcflg': '', 'rn': '', 'rnQcflg': '', 'ws': '1.9', 'wsQcflg': '', 'wd': '110', 'wdQcflg': '', 'hm': '93', 'hmQcflg': '', 'pv': '4.2', 'td': '-5.0', 'pa': '1025.0', 'paQcflg': '', 'ps': '1030.2', 'psQcflg': '', 'ss': '', 'ssQcflg

In [8]:
# df = pd.DataFrame()
# df['tim'] = res['tm']        #시간
# df['temp'] = res['ta']        #기온
# df['humid'] = res['hm']       #습도
# df['rain'] = res['rn']        #강수량
# df['wind'] = res['wd']        #풍속
# df['sunshine'] = res['ss']    #일조:태양이 지표면을 비친 시간
# df['solar radiation'] = res['icsr'] #일사:지표면에 도달한 태양복사에너지

# df

,tim,temp,humid,rain,wind,sunshine,solar radiation
0,2023-01-01 01:00,-4.5,94,,140,,
1,2023-01-01 02:00,-4.1,93,,110,,
2,2023-01-01 03:00,-4.4,93,,110,,
3,2023-01-01 04:00,-4.9,94,,50,,
4,2023-01-01 05:00,-5.3,98,,0,,
5,2023-01-01 06:00,-4.9,96,,250,,
6,2023-01-01 07:00,-6.2,99,,270,,
7,2023-01-01 08:00,-5.6,100,,290,0.0,0.0
8,2023-01-01 09:00,-4.1,95,,0,0.9,0.43
9,2023-01-01 10:00,-0.4,63,,290,1.0,1.22
